In [2]:
! python -m pip install "ensembl_lite @ git+https://github.com/cogent3/EnsemblLite.git@develop"

  Cloning https://github.com/cogent3/EnsemblLite.git (to revision develop) to /private/var/folders/d8/pdrt51hx2jb17vf6k28_x6mh0000gn/T/pip-install-jyka93cg/ensembl-lite_6eca0332e1264409a629af9f6dbdcdaa
  Running command git clone --filter=blob:none --quiet https://github.com/cogent3/EnsemblLite.git /private/var/folders/d8/pdrt51hx2jb17vf6k28_x6mh0000gn/T/pip-install-jyka93cg/ensembl-lite_6eca0332e1264409a629af9f6dbdcdaa
  Resolved https://github.com/cogent3/EnsemblLite.git to commit 31c8fd276266930801c7d569e0c0517598cc7aa3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/cogent3/cogent3.git (to revision develop) to /private/var/folders/d8/pdrt51hx2jb17vf6k28_x6mh0000gn/T/pip-install-jyka93cg/cogent3_f32628dc7d3041018018a3dce68a208b
  Running command git clone --filter=blob:none --quiet https://github.com/cogent3/cogent3.git /private/var/folders/d8/pdrt51hx2jb17vf6k28_x6mh0

In [3]:
import pathlib
from ensembl_lite._download import get_ensembl_trees, download_ensembl_tree
from ensembl_lite._species import Species
from cogent3 import load_tree

In [205]:
kwargs = dict(host="ftp.ensembl.org", remote_path="pub", release="111")
trees = get_ensembl_trees(**kwargs)
trees


['pub/release-111/compara/species_trees/100_mammals_Cactus_default.nh',
 'pub/release-111/compara/species_trees/10_primates_EPO_default.nh',
 'pub/release-111/compara/species_trees/16_pig_breeds_EPO-Extended_default.nh',
 'pub/release-111/compara/species_trees/17_sauropsids_EPO_default.nh',
 'pub/release-111/compara/species_trees/21_murinae_EPO_default.nh',
 'pub/release-111/compara/species_trees/24_primates_EPO-Extended_default.nh',
 'pub/release-111/compara/species_trees/27_sauropsids_EPO-Extended_default.nh',
 'pub/release-111/compara/species_trees/39_fish_EPO_default.nh',
 'pub/release-111/compara/species_trees/43_eutherian_mammals_EPO_default.nh',
 'pub/release-111/compara/species_trees/63_amniota_vertebrates_Mercator-Pecan_default.nh',
 'pub/release-111/compara/species_trees/65_fish_EPO-Extended_default.nh',
 'pub/release-111/compara/species_trees/91_eutherian_mammals_EPO-Extended_default.nh',
 'pub/release-111/compara/species_trees/default_ncRNA-trees_cafe.nh',
 'pub/release-111

In [206]:
# Filter the list by excluding certain indices
exclude_indices = set(range(12, 21)) | {22}
DNA_trees = [tree for i, tree in enumerate(trees) if i not in exclude_indices]


In [208]:
for i in range(len(trees)):
    tree_name = pathlib.Path(trees[i]).name
    download_kwargs = kwargs.copy()
    download_kwargs["tree_fname"] = tree_name

    tree = download_ensembl_tree(**download_kwargs)

    outpath = f"../data/ensemble_trees/{tree_name}"
    tree.write(outpath)



In [175]:
def find_common_name_and_unknown_species(tree):
    ## looking up common name from latin name
    tree_name = pathlib.Path(tree).name
    download_kwargs = kwargs.copy()
    download_kwargs["tree_fname"] = tree_name

    tree = download_ensembl_tree(**download_kwargs)
    tip_names = tree.get_tip_names()

    # Assume tree and Species are already defined and imported correctly
    tip_names = tree.get_tip_names()
    common_names = {}
    unknown_species = {}
    # Loop through all tip names to get common names
    for name in tip_names:
        try:
            common_name = Species.get_common_name(name)
            common_names[name] = common_name
        except ValueError:
            unknown_species[name] = 'Unknown'
            common_names[name] = 'Unknown'
    return common_names, unknown_species


In [194]:
common_names_all = {}
unknown_species_all = {}
for tree in DNA_trees:
    dict_key = pathlib.Path(tree).name
    common_name_, unknown_species_ = find_common_name_and_unknown_species(tree)
    common_names_all[dict_key] = common_name_
    unknown_species_all[dict_key] = unknown_species_



In [181]:
unknown_species_all

{'100_mammals_Cactus_default.nh': {'Gallus_gallus_reference_breed': 'Unknown',
  'Anolis_carolinensis_reference_strain': 'Unknown',
  'Bos_indicus_x_Bos_taurus': 'Unknown',
  'Bos_taurus_reference_breed': 'Unknown',
  'Capra_hircus_reference_breed': 'Unknown',
  'Ovis_aries_reference_breed': 'Unknown',
  'Sus_scrofa_reference_breed': 'Unknown',
  'Equus_caballus_breed_thoroughbred': 'Unknown',
  'Equus_asinus_reference_breed': 'Unknown',
  'Neogale_vison': 'Unknown',
  'Gorilla_gorilla_gorilla': 'Unknown',
  'Heterocephalus_glaber_reference_strain': 'Unknown',
  'Mus_musculus_reference_CL57BL6_strain': 'Unknown',
  'Rattus_norvegicus_strain_BN/NHsdMcwi': 'Unknown',
  'Cricetulus_griseus_strain_17A/GY': 'Unknown'},
 '10_primates_EPO_default.nh': {'Pan_troglodytes_Pan_tro_3.0': 'Unknown',
  'Homo_sapiens_GRCh38': 'Unknown',
  'Gorilla_gorilla_gorilla_gorGor4': 'Unknown',
  'Pongo_abelii_Susie_PABv2': 'Unknown',
  'Nomascus_leucogenys_Nleu_3.0': 'Unknown',
  'Macaca_mulatta_Mmul_10': 'Unk

In [195]:
unknown_species_list = set()
for tree in unknown_species_all.keys():
    for species in unknown_species_all[tree]:
        unknown_species_list.add(species)

In [196]:
unknown_species = {key: "unknown" for key in unknown_species_list}

unknown_species = {'Ailuropoda_melanoleuca_reference_isolate': 'Giant Panda',
 'Anabas_testudineus_fAnaTes1.2': 'Climbing Perch_fAnaTes1.2',
 'Anolis_carolinensis_reference_strain': 'Green Anole',
 'Astyanax_mexicanus_Astyanax_mexicanus-2.0': 'Mexican tetra',
 'Balaenoptera_musculus_mBalMus1.v2': 'Blue whale_mBalMus1.v2',
 'Bos_grunniens_LU_Bosgru_v3.0': 'Yak',
 'Bos_indicus_x_Bos_taurus': 'India Cattle x Europe Cattle',
 'Bos_indicus_x_Bos_taurus_UOA_Brahman_1': 'India Cattle x Europe Cattle__UOA_Brahman_1',
 'Bos_taurus_reference_breed': 'Europe Cattle',
 'Caenorhabditis_elegans_strain_N2': "The worm_strain_N2",
 'Callithrix_jacchus_mCalJac1.pat.X': 'Common marmoset_mCalJac1.pat.X',
 'Camelus_dromedarius_CamDro2': 'Arabian Camel',
 'Canis_lupus_dingo_ASM325472v1': 'Dingo_ASM325472v1',
 'Canis_lupus_familiaris_reference_breed': 'Wolf',
 'Capra_hircus_reference_breed': 'Goat',
 'Carlito_syrichta_Tarsius_syrichta-2.0.1': 'Philippine tarsier-2.0.1',
 'Catagonus_wagneri_CatWag_v2_BIUU_UCD': 'Chacoan peccary',
 'Cavia_porcellus_Cavpor3.0': 'Guinea pi_Cavpor3.0',
 'Cebus_imitator_Cebus_imitator-1.0': 'Panamanian white-faced capuchin',
 'Cervus_hanglu_yarkandensis_CEY_v1': 'Yarkand deer',
 'Chlorocebus_sabaeus_ChlSab1.1': 'Green monkey',
 'Cricetulus_griseus': 'Chinese hamster',
 'Cricetulus_griseus_CHOK1GS_HDv1': 'Chinese hamster_CHOK1GS_HDv1',
 'Cricetulus_griseus_strain_17A/GY': 'Chinese hamster_strain_17A/GY',
 'Danio_rerio_GRCz11': 'Zebrafish_GRCz11',
 'Dasypus_novemcinctus_Dasnov3.0': 'Nine-banded armadillo_Dasnov3.0',
 'Delphinapterus_leucas_ASM228892v3': 'Beluga whale__ASM228892v3',
 'Dicentrarchus_labrax_dlabrax2021': 'European seabass_dlabrax2021',
 'Dipodomys_ordii_Dord_2.0': 'Ord`s kangaroo rat',
 'Equus_asinus_reference_breed': 'African wild ass',
 'Equus_caballus_breed_thoroughbred': 'Thoroughbred',
 'Esox_lucius_Eluc_v4': 'Northern pikeEluc_v4',
 'Felis_catus_Felis_catus_9.0': 'Cat_9.0',
 'Ficedula_albicollis_FicAlb1.5': 'Collared flycatcher_FicAlb1.5',
 'Gadus_morhua_gadMor3.0': 'Atlantic cod_gadMor3.0',
 'Gallus_gallus_reference_breed': 'Chicken',
 'Gorilla_gorilla_gorilla': 'Western lowland gorilla',
 'Gorilla_gorilla_gorilla_gorGor4': 'Western lowland gorilla_gorGor4',
 'Heterocephalus_glaber_reference_strain': 'Naked mole-rat',
 'Homo_sapiens_GRCh38': 'Human_GRCh38',
 'Ictidomys_tridecemlineatus_SpeTri2.0': 'Thirteen-lined ground squirrel_SpeTri2.0',
 'Loxodonta_africana_loxAfr3': 'African bush elephant_loxAfr3',
 'Macaca_fascicularis_Macaca_fascicularis_6.0': 'Crab-eating macaque_6.0',
 'Macaca_mulatta_Mmul_10': 'Indochinese rhesus macaque_Mmul_10',
 'Marmota_marmota_marmota_marMar2.1': 'Alpine marmot_marMar2.1',
 'Mastacembelus_armatus_fMasArm1.2': 'Zig-zag eel_fMasArm1.2',
 'Meleagris_gallopavo_reference_strain': 'Wild turkey',
 'Microcebus_murinus_Mmur_3.0': 'Gray mouse lemur_Mmur_3.0',
 'Microtus_ochrogaster_MicOch1.0': 'Prairie vole_MicOch1.0',
 'Monodon_monoceros_NGI_Narwhal_1': 'Narwhal',
 'Mus_caroli_strain_CAROLI_EIJ': 'Ryukyu mouse_strain_CAROLI_EIJ',
 'Mus_musculus_castaneus_strain_CAST/EiJ': 'southeastern Asian house mouse_strain_CAST/EiJ',
 'Mus_musculus_domesticus_strain_WSB/EiJ': 'House mouse_strain_WSB/EiJ',
 'Mus_musculus_musculus_strain_PWK/PhJ': 'House mouse_musculus_strain_PWK/PhJ',
 'Mus_musculus_reference_CL57BL6_strain': 'House mouse_reference_CL57BL6_strain',
 'Mus_musculus_strain_129S1/SvImJ': 'House mouse_strain_129S1/SvImJ',
 'Mus_musculus_strain_A/J': 'House mouse_strain_A/J',
 'Mus_musculus_strain_AKR/J': 'House mouse_strain_AKR/J',
 'Mus_musculus_strain_BALB/cJ': 'House mouse_strain_BALB/cJ',
 'Mus_musculus_strain_C3H/HeJ': 'House mouse_strain_C3H/HeJ',
 'Mus_musculus_strain_C57BL/6NJ': 'House mouse_strain_C57BL/6NJ',
 'Mus_musculus_strain_CBA/J': 'House mouse_strain_CBA/J',
 'Mus_musculus_strain_DBA/2J': 'House mouse_strain_DBA/2J',
 'Mus_musculus_strain_FVB/NJ': 'House mouse_strain_FVB/NJ',
 'Mus_musculus_strain_LP/J': 'House mouse_strain_LP/J',
 'Mus_musculus_strain_NOD/ShiLtJ': 'House mouse_strain_NOD/ShiLtJ',
 'Mus_musculus_strain_NZO/HlLtJ': 'House mouse_strain_NZO/HlLtJ',
 'Mus_pahari_strain_PAHARI_EIJ': 'House mouse_strain_PAHARI_EIJ',
 'Mus_spretus_strain_SPRET/EiJ': 'House mouse_strain_SPRET/EiJ',
 'Myotis_lucifugus_Myoluc2.0': 'Little brown bat_Myoluc2.0',
 'Neogale_vison': 'American mink',
 'Nomascus_leucogenys_Nleu_3.0': 'Northern white-cheeked gibbon_Nleu_3.0',
 'Ochotona_princeps_OchPri2.0-Ens': 'American pika_OchPri2.0-Ens',
 'Oncorhynchus_mykiss_USDA_OmykA_1.1': 'Rainbow trout_USDA_OmykA_1.1',
 'Oreochromis_niloticus_O_niloticus_UMD_NMBU': 'Nile tilapia',
 'Oryctolagus_cuniculus_OryCun2.0': 'European rabbit_OryCun2.0',
 'Oryzias_latipes_ASM223467v1': 'Japanese rice fish_ASM223467v1',
 'Otolemur_garnettii_OtoGar3': 'Northern greater galago',
 'Ovis_aries_reference_breed': 'Sheep',
 'Pan_paniscus_panpan1.1': 'Bonobo_panpan1.1',
 'Pan_troglodytes_Pan_tro_3.0': 'Chimpanzee_Pan_tro_3.0',
 'Panthera_leo_PanLeo1.0': 'Lion_PanLeo1.0',
 'Panthera_pardus_PanPar1.0': 'Leopard_PanPar1.0',
 'Papio_anubis_Panubis1.0': 'Olive baboon_Panubis1.0',
 'Peromyscus_maniculatus_bairdii_HU_Pman_2.1': 'Prairie Deermouse_HU_Pman_2.1',
 'Phocoena_sinus_mPhoSin1.pri': 'Vaquita_mPhoSin1.pri',
 'Physeter_catodon_ASM283717v2': 'Sperm whale_ASM283717v2',
 'Pongo_abelii_Susie_PABv2': 'Sumatran orangutan_Susie_PABv2',
 'Procavia_capensis_proCap1': 'Rock hyrax_proCap1',
 'Pteropus_vampyrus_pteVam1': 'Large flying fox_pteVam1',
 'Rattus_norvegicus_strain_BN/NHsdMcwi': 'Brown rat_strain_BN/NHsdMcwi',
 'Rhinolophus_ferrumequinum_mRhiFer1_v1.p': 'Greater horseshoe bat_mRhiFer1_v1.p',
 'Saccharomyces_cerevisiae_S288C': 'Yeast_S288C',
 'Salmo_salar_Ssal_v3.1': 'Atlantic salmon_Ssal_v3.1',
 'Sciurus_vulgaris_mSciVul1.1': 'Red squirre_mSciVul1.1',
 'Scleropages_formosus_fSclFor1.1': 'Asian arowana_fSclFor1.1',
 'Scophthalmus_maximus_reference_strain': 'Turbot',
 'Sorex_araneus_COMMON_SHREW1': 'Common shrew',
 'Sus_scrofa_breed_bamei': 'Pig_breed_bamei',
 'Sus_scrofa_breed_berkshire': 'Pig_breed_berkshire',
 'Sus_scrofa_breed_hampshire': 'Pig_breed_hampshire',
 'Sus_scrofa_breed_jinhua': 'Pig_breed_jinhua',
 'Sus_scrofa_breed_landrace': 'Pig_breed_landrace',
 'Sus_scrofa_breed_largewhite': 'Pig__breed_largewhite',
 'Sus_scrofa_breed_meishan': "Pig_breed_meishan",
 'Sus_scrofa_breed_pietrain': 'Pig__breed_pietrain',
 'Sus_scrofa_breed_rongchang': 'Pig_breed_rongchang',
 'Sus_scrofa_breed_tibetan': 'Pig_breed_tibetan',
 'Sus_scrofa_breed_usmarc': 'Pig_breed_usmarc',
 'Sus_scrofa_breed_wuzhishan': 'Pig_breed_wuzhishan',
 'Sus_scrofa_reference_breed': 'Pig_reference_breed',
 'Taeniopygia_guttata_bTaeGut1_v1.p': 'Australian zebra finch_bTaeGut1_v1.p',
 'Takifugu_rubripes_fTakRub1.2': 'Japanese puffer_fTakRub1.2',
 'Tetraodon_nigroviridis_TETRAODON8': 'Puffer fish_TETRAODON8',
 'Tursiops_truncatus_turTru1': 'Common bottlenose dolphin_turTru1',
 'Vicugna_pacos_vicPac1': 'Alpaca_vicPac1',
 'Xiphophorus_maculatus_X_maculatus-5.0-male': 'Southern platyfishX_maculatus-5.0-male'}



In [197]:
# Loop through each tree in common_name_all
for tree, species_dict in common_names_all.items():
    # Loop through each species in the current tree
    for species, common_name in species_dict.items():
        # Check if the common name is 'Unknown'
        if common_name == 'Unknown':
            # If yes, update it with the corresponding common name from unknown_species
            if species in unknown_species:
                species_dict[species] = unknown_species[species]

In [198]:
common_names_all['43_eutherian_mammals_EPO_default.nh']

{'Cricetulus_griseus_CHOK1GS_HDv1': 'Chinese hamster_CHOK1GS_HDv1',
 'Microtus_ochrogaster_MicOch1.0': 'Prairie vole_MicOch1.0',
 'Peromyscus_maniculatus_bairdii_HU_Pman_2.1': 'Prairie Deermouse_HU_Pman_2.1',
 'Mus_spretus_strain_SPRET/EiJ': 'House mouse_strain_SPRET/EiJ',
 'Mus_musculus_reference_CL57BL6_strain': 'House mouse_reference_CL57BL6_strain',
 'Mus_caroli_strain_CAROLI_EIJ': 'Ryukyu mouse_strain_CAROLI_EIJ',
 'Mus_pahari_strain_PAHARI_EIJ': 'House mouse_strain_PAHARI_EIJ',
 'Rattus_norvegicus_strain_BN/NHsdMcwi': 'Brown rat_strain_BN/NHsdMcwi',
 'Cavia_porcellus_Cavpor3.0': 'Guinea pi_Cavpor3.0',
 'Sciurus_vulgaris_mSciVul1.1': 'Red squirre_mSciVul1.1',
 'Marmota_marmota_marmota_marMar2.1': 'Alpine marmot_marMar2.1',
 'Oryctolagus_cuniculus_OryCun2.0': 'European rabbit_OryCun2.0',
 'Pan_troglodytes_Pan_tro_3.0': 'Chimpanzee_Pan_tro_3.0',
 'Pan_paniscus_panpan1.1': 'Bonobo_panpan1.1',
 'Homo_sapiens_GRCh38': 'Human_GRCh38',
 'Gorilla_gorilla_gorilla_gorGor4': 'Western lowland

In [199]:
paths = list(pathlib.Path("../data/ensemble_trees").glob("**/*.nh"))
paths

[PosixPath('../data/ensemble_trees/43_eutherian_mammals_EPO_default.nh'),
 PosixPath('../data/ensemble_trees/24_primates_EPO-Extended_default.nh'),
 PosixPath('../data/ensemble_trees/65_fish_EPO-Extended_default.nh'),
 PosixPath('../data/ensemble_trees/21_murinae_EPO_default.nh'),
 PosixPath('../data/ensemble_trees/17_sauropsids_EPO_default.nh'),
 PosixPath('../data/ensemble_trees/16_pig_breeds_EPO-Extended_default.nh'),
 PosixPath('../data/ensemble_trees/100_mammals_Cactus_default.nh'),
 PosixPath('../data/ensemble_trees/10_primates_EPO_default.nh'),
 PosixPath('../data/ensemble_trees/vertebrates_species-tree_Ensembl.nh'),
 PosixPath('../data/ensemble_trees/27_sauropsids_EPO-Extended_default.nh'),
 PosixPath('../data/ensemble_trees/39_fish_EPO_default.nh'),
 PosixPath('../data/ensemble_trees/91_eutherian_mammals_EPO-Extended_default.nh'),
 PosixPath('../data/ensemble_trees/63_amniota_vertebrates_Mercator-Pecan_default.nh')]

In [203]:
path = paths[0]
print(path)
tree_10primate= load_tree(path, format=None, underscore_unmunge=False)
tree_10primate.get_figure(show_support=True, threshold=0.8)

../data/ensemble_trees/43_eutherian_mammals_EPO_default.nh
